# ARPS Curve Fix Validation

This notebook validates the fix for the ARPS curve fitting bug by:
1. Re-running the curve fitting with the corrected time indexing
2. Visualizing the results to confirm proper alignment
3. Comparing R² values before and after

In [ ]:
# Setup
import sys
sys.path.insert(0, '.')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from AnalyticsAndDBScripts.csv_loader import CSVDataLoader
from AnalyticsAndDBScripts.visualization_utils import plot_decline_curve
import AnalyticsAndDBScripts.prod_fcst_functions as fcst

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Imports successful!")

In [ ]:
# Load your CSV data
csv_loader = CSVDataLoader('sample_production_data.csv', 'sample_well_list.csv')

# Get a sample well's production data
wellid = 12345678901
measure = 'GAS'
last_prod_date = pd.Timestamp('2024-10-01')

actual_data = csv_loader.get_well_production(
    wellid=wellid,
    measure=measure,
    last_prod_date=last_prod_date,
    fit_months=60
)

print(f"Loaded {len(actual_data)} months of production data")
print(f"Date range: {actual_data['Date'].min()} to {actual_data['Date'].max()}")
display(actual_data.head())

In [ ]:
# Prepare data for fitting (with CORRECTED time indexing)
df = actual_data.copy()

# Remove first row for noise reduction (as done in the fitting code)
if len(df) > 1:
    df = df.iloc[1:].reset_index(drop=True)

# CORRECTED: Zero-based time indexing
t_act = df['Date'].rank(method='min', ascending=True).to_numpy() - 1
q_act = df['Value'].to_numpy()

print(f"Time array (first 10 values): {t_act[:10]}")
print(f"Production array (first 10 values): {q_act[:10]}")
print(f"\nNote: Time starts at 0 (CORRECT for ARPS equations)")

In [ ]:
# Fit ARPS curve with corrected time indexing
Qi_guess = np.max(q_act)
Dei_init = 0.15  # 15% initial decline
Dei_min = 0.06   # 6% minimum (terminal decline for gas)
Dei_max = 0.30   # 30% maximum
b_guess = 0.9
b_min = 0.5
b_max = 1.4
Def = 0.06  # Terminal decline for gas

# Configure fitting
bounds = ((Qi_guess*0.9, Dei_min, b_min), (Qi_guess, Dei_max, b_max))
initial_guess = [Qi_guess, Dei_init, b_guess]
config = {
    'optimize': ['Qi', 'Dei', 'b'],
    'fixed': {'Def': Def}
}

print("Fitting ARPS curve with corrected time indexing...")
result = fcst.perform_curve_fit(
    t_act, q_act, initial_guess, bounds, 
    config, method='curve_fit', trials=1000
)

# Handle different return formats
if isinstance(result, tuple):
    optimized_params = result[0]
else:
    optimized_params = result

qi_fit, Dei_fit, b_fit = optimized_params

print(f"\n✅ Fitting complete!")
print(f"Qi (Initial Rate): {qi_fit:.2f} MCF/day")
print(f"Dei (Initial Decline): {Dei_fit:.1%} per year")
print(f"b-factor: {b_fit:.3f}")

In [ ]:
# Calculate fitted curve and R²
q_pred = fcst.varps_decline(1, 1, qi_fit, Dei_fit, Def, b_fit, t_act, 0, 0)[3]
r_squared, rmse, mae = fcst.calc_goodness_of_fit(q_act, q_pred)

print(f"Goodness of Fit Metrics:")
print(f"  R² = {r_squared:.4f}")
print(f"  RMSE = {rmse:.2f}")
print(f"  MAE = {mae:.2f}")

In [ ]:
# Visualize the fit
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Generate forecast (24 months ahead)
forecast_months = 24
t_forecast = np.arange(0, len(t_act) + forecast_months)
q_forecast = fcst.varps_decline(1, 1, qi_fit, Dei_fit, Def, b_fit, t_forecast, 0, 0)[3]

# Create date arrays for plotting
start_date = actual_data['Date'].min()
all_dates = pd.date_range(start=start_date, periods=len(t_forecast), freq='MS')
history_dates = all_dates[:len(t_act)]
forecast_dates = all_dates[len(t_act):]

# Plot 1: Linear scale
ax1.plot(actual_data['Date'], actual_data['Value'], 'o', 
         label='Actual Production', markersize=8, color='#2E86AB', alpha=0.7)
ax1.plot(history_dates, q_forecast[:len(t_act)], '-', 
         label='ARPS Fit (Corrected)', linewidth=3, color='#A23B72')
ax1.plot(forecast_dates, q_forecast[len(t_act):], '--', 
         label='Forecast', linewidth=3, color='#F18F01', alpha=0.8)
ax1.axvline(x=actual_data['Date'].max(), color='gray', linestyle=':', linewidth=2, alpha=0.5)

ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel(f'{measure} Rate (MCF/day)', fontsize=12)
ax1.set_title(
    f'Well {wellid} - {measure} ARPS Decline Curve (CORRECTED)\n'
    f'R² = {r_squared:.4f} | Qi = {qi_fit:.1f} | Dei = {Dei_fit:.3f} | b = {b_fit:.3f}', 
    fontsize=14, fontweight='bold'
)
ax1.legend(fontsize=11, loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: Log scale
ax2.semilogy(actual_data['Date'], actual_data['Value'], 'o', 
             label='Actual Production', markersize=8, color='#2E86AB', alpha=0.7)
ax2.semilogy(history_dates, q_forecast[:len(t_act)], '-', 
             label='ARPS Fit (Corrected)', linewidth=3, color='#A23B72')
ax2.semilogy(forecast_dates, q_forecast[len(t_act):], '--', 
             label='Forecast', linewidth=3, color='#F18F01', alpha=0.8)
ax2.axvline(x=actual_data['Date'].max(), color='gray', linestyle=':', linewidth=2, alpha=0.5)

ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel(f'{measure} Rate (log scale)', fontsize=12)
ax2.set_title('Log Scale View - Exponential Decline Pattern', fontsize=12, fontweight='bold')
ax2.legend(fontsize=11, loc='upper right')
ax2.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('arps_validation_corrected.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Plot saved as 'arps_validation_corrected.png'")

In [ ]:
# Compare: Show what the WRONG time indexing would have produced
print("\n" + "="*60)
print("COMPARISON: Wrong vs Correct Time Indexing")
print("="*60)

# WRONG: Time starting at 1
t_act_wrong = df['Date'].rank(method='min', ascending=True).to_numpy()
print(f"\nWRONG time array (first 5): {t_act_wrong[:5]}")
print(f"CORRECT time array (first 5): {t_act[:5]}")

# Fit with wrong time indexing
result_wrong = fcst.perform_curve_fit(
    t_act_wrong, q_act, initial_guess, bounds, 
    config, method='curve_fit', trials=1000
)
if isinstance(result_wrong, tuple):
    params_wrong = result_wrong[0]
else:
    params_wrong = result_wrong

qi_wrong, Dei_wrong, b_wrong = params_wrong
q_pred_wrong = fcst.varps_decline(1, 1, qi_wrong, Dei_wrong, Def, b_wrong, t_act_wrong, 0, 0)[3]
r2_wrong, _, _ = fcst.calc_goodness_of_fit(q_act, q_pred_wrong)

print(f"\nWRONG Indexing Results:")
print(f"  Qi = {qi_wrong:.2f}, Dei = {Dei_wrong:.3f}, b = {b_wrong:.3f}, R² = {r2_wrong:.4f}")

print(f"\nCORRECT Indexing Results:")
print(f"  Qi = {qi_fit:.2f}, Dei = {Dei_fit:.3f}, b = {b_fit:.3f}, R² = {r_squared:.4f}")

print(f"\nImprovement in R²: {(r_squared - r2_wrong):.4f} ({((r_squared - r2_wrong)/r2_wrong * 100):.1f}% better)")

In [ ]:
# Side-by-side comparison plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# LEFT: Wrong time indexing
t_viz = np.arange(0, len(t_act))  # Visualization always uses 0-based
q_viz_wrong = fcst.varps_decline(1, 1, qi_wrong, Dei_wrong, Def, b_wrong, t_viz, 0, 0)[3]

ax1.plot(actual_data['Date'], actual_data['Value'], 'o', 
         label='Actual', markersize=8, color='#2E86AB', alpha=0.7)
ax1.plot(history_dates, q_viz_wrong, '-', 
         label='ARPS Fit (WRONG)', linewidth=3, color='red')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel(f'{measure} Rate', fontsize=12)
ax1.set_title(f'WRONG Time Indexing (t starts at 1)\nR² = {r2_wrong:.4f}', 
              fontsize=13, fontweight='bold', color='red')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# RIGHT: Correct time indexing
ax2.plot(actual_data['Date'], actual_data['Value'], 'o', 
         label='Actual', markersize=8, color='#2E86AB', alpha=0.7)
ax2.plot(history_dates, q_forecast[:len(t_act)], '-', 
         label='ARPS Fit (CORRECT)', linewidth=3, color='green')
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylabel(f'{measure} Rate', fontsize=12)
ax2.set_title(f'CORRECT Time Indexing (t starts at 0)\nR² = {r_squared:.4f}', 
              fontsize=13, fontweight='bold', color='green')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('arps_comparison_wrong_vs_correct.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Comparison plot saved as 'arps_comparison_wrong_vs_correct.png'")

## Validation Summary

The corrected time indexing (starting at t=0) ensures:
1. **Proper alignment** between fitted curve and actual data
2. **Correct interpretation** of Qi as the initial rate at t=0
3. **Improved R² values** indicating better fit quality
4. **Consistency** with standard ARPS practice in petroleum engineering

### Key Takeaway
The bug was causing the curve to be fitted with t=[1,2,3,...] but visualized with t=[0,1,2,...], resulting in a systematic offset. The fix ensures both use t=[0,1,2,...] consistently.